In [89]:
class ListNode:
    def __init__(self, key, value, prev=None, next=None):
        self.key = key 
        self.value = value
        self.prev = prev
        self.next = next
        
    def delete(self):
        if self.prev:
            self.prev.next = self.next
        if self.next:
            self.next.prev = self.prev

class DoublyLinkedList:
    def __init__(self, node=None):
        self.head = node
        self.tail = node
        self.length = 1 if node is not None else 0

    def __len__(self):
        return self.length

    def add_to_head(self, key, value):
        new_node = ListNode(key, value)
        self.length += 1
        if not self.head and not self.tail:
            self.head = new_node
            self.tail = new_node
        else:
            new_node.next = self.head
            self.head.prev = new_node
            self.head = new_node

    def remove_from_head(self):
        key, value = self.head.key, self.head.value
        self.delete(self.head)
        return key, value

    def add_to_tail(self, key, value):
        new_node = ListNode(key, value)
        self.length += 1
        if not self.head and not self.tail:
            self.head = new_node
            self.tail = new_node
        else:
            new_node.prev = self.tail
            self.tail.next = new_node
            self.tail = new_node

    def remove_from_tail(self):
        key, value = self.tail.key, self.tail.value
        self.delete(self.tail)
        return key, value

    def move_to_front(self, node):
        if node is self.head:
            return
        self.add_to_head(node.key, node.value)
        self.delete(node)

    def move_to_end(self, node):
        if node is self.tail:
            return
        self.add_to_tail(node.key, node.value)
        self.delete(node)

    def delete(self, node):
        self.length -= 1
        if self.head is self.tail:
            self.head = None
            self.tail = None
        elif node is self.head:
            self.head = node.next
            node.delete()
        elif node is self.tail:
            self.tail = node.prev
            node.delete()
        else:
            node.delete()

    def get_max(self):
        current = self.head
        max = self.head.value

        while current is not None:
            if current.value > max:
                max = current.value
            current = current.next
        return max

In [92]:
# class HashTableEntry:
#     def __init__(self, key, value):
#         self.key = key
#         self.value = value
#         self.next = None

class HashTable:
    def __init__(self, CAPACITY):
        self.capacity = CAPACITY 
        self.size = 0 
        self.storage = []
        
        for x in range(0, self.capacity):
            DLL = DoublyLinkedList()
            self.storage.append(DLL)
        
    def djb2(self, key):
        hash = 5381
        for x in key:
            hash = (( hash << 5) + hash) + ord(x)
        return hash & 0xFFFFFFFF

    def hash_index(self, key):
        return self.djb2(key) % self.capacity

    def put(self, key, value):
        self.size += 1 
        index = self.hash_index(key)
        DLL = self.storage[index]
        DLL.add_to_head(key, value)

    def delete(self, key):
        index = self.hash_index(key)
        node = self.storage[index].head
        while node.key != key:
            node = node.next 
        DLL = self.storage[index]   
        DLL.remove_from_head()
               
    def get(self, key):
        index = self.hash_index(key)
        node = self.storage[index].head
        while node.key != key:
            node = node.next 
        return node.value     
           
    def resize(self):
        old_storage = self.storage
        self.storage = []
        new_cap = self.capacity*2
        
        for x in range(0, new_cap):
            DLL = DoublyLinkedList()
            self.storage.append(DLL)
            
        for DLL in old_storage:
            node = DLL.head
            while node:
                key = node.key
                value = node.value
                node = node.next
                self.put(key, value)

In [93]:
ht = HashTable(2)

ht.put("line_1", "Tiny hash table")
ht.put("line_2", "Filled beyond capacity")
ht.put("line_3", "Linked list saves the day!")

print("")

# Test storing beyond capacity
print(ht.get("line_1"))
print(ht.get("line_2"))
print(ht.get("line_3"))

# Test resizing
old_capacity = len(ht.storage)
ht.resize()
new_capacity = len(ht.storage)

print(f"\nResized from {old_capacity} to {new_capacity}.\n")

# Test if data intact after resizing
print(ht.get("line_1"))
print(ht.get("line_2"))
print(ht.get("line_3"))

print("")


Tiny hash table
Filled beyond capacity
Linked list saves the day!

Resized from 2 to 4.

Tiny hash table
Filled beyond capacity
Linked list saves the day!



In [94]:
ht = HashTable(2)

ht.put("line_1", "Tiny hash table")
ht.put("line_2", "Filled beyond capacity")
ht.put("line_3", "Linked list saves the day!")

print(ht.get("line_1"))
print(ht.get("line_2"))
print(ht.get("line_3"))

Tiny hash table
Filled beyond capacity
Linked list saves the day!


open addressing:
linear probing involves putting a would be collision on an open space in the hash table 
and relies on increasing the size of the hash table to accomodate more hashes? 

put:
scan forward from the hash index until we find either the key, or None
If we find a deleted slot along the way, keep it in mind
put the value there

get:
scan forward from the hash index until we find either the key, or None
put the value there

delete:
scan forward from the hash index until we find either the key, or None
If we find the key mark it as deleted

Collison Resolution by Chaining

Each slot of the hash table holds a linked list
Collisons are handled by adding multiple items to the same linked list

Slot
Index (linked list)

0 -> None
1 -> ("foo", 12) 
2 -> ("baz", 999)
3 -> None

put ("foo", 12) # hashes to index 1 
put ("bar", 30) # hashes to index 2 
put ("baz", 999) # hashes to index 2 <<

get("beej") # hashes to index 1 
return None
get("baz") # hashes to index 2
return 999

delete("bar") # hashes to index 2 


Put:
Find the hash index
Search the list for the key
If it's there, replace the value
If it's not, append a new record to the list 

get("beej")
Get:
Find the hash index
Search the list for the key 
If found return the value
Else return None 

Delete:
Find the hash index
Search the list for the key 
If found, delete the node from the list
Else return None 


linked lists refresher 

"""

(1) -> (2) -> (3) -> None

^
head 

class Node:
    def __init__(self, value):
        self.value = value
        self.next = None 
        
        
        
head = None 



def insert_at_head(value:
    n = None(value)
    n.next = head

In [ ]:
# keep track of load factor
# increment item count each time an item is put into the table 

In [ ]:
# resizing, using load factor to automatically resize 

In [ ]:
# step one make a new table
# step two go through all the elements and has into new list 

In [ ]:
# automatic resizing

when before you put, if load > 0.7:
    double size of the hastable

when after you delete, if load < 0.2:
    halve the size of the hash table
        down to the minimum at most 

In [ ]:
# HAVE TO MAKE A NEW ARRAY OF BUCKETS
# PROBLEM SOLVED 
# everything else can stay in place no need to delete 

In [ ]:
what do you mean some hash functions are tuned for using with hash tables? 